In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import PIL 
from PIL import Image

print(tf.__version__)

2.4.1


In [2]:
img_height = 28
img_width = 28

In [3]:
model = keras.Sequential([
    layers.Input((28,28,1)), #28 * 28 pixels, only 1 color (greyscale)
    layers.Dense(128, activation="selu"), #128 nodes,  activation = rectify linear unit, fast activation function
    layers.Conv2D(16,3, padding='same'), #2d convulution layer
    layers.Conv2D(32,3, padding='same'),
    layers.MaxPooling2D(),#downsamples input so that feature maps can be created. usually added after convulution layer
    layers.Flatten(), #[[1],[2],[3]] --> [1,2,3]
    layers.Dense(3), #representing 3 classes, output layer
])

In [4]:
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    'C:/Users/evanp/AI/Capstone/test_',
    labels='inferred',
    label_mode = "int",
    color_mode='grayscale',
    image_size=(img_height, img_width),
    shuffle=True,
    seed=123, #same seed results in same validation test, good for testing bad for production
    validation_split=0.5,
    subset="training",
)

Found 543 files belonging to 3 classes.
Using 272 files for training.


In [5]:
ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
    'C:/Users/evanp/AI/Capstone/test_',
    labels='inferred',
    label_mode = "int",
    color_mode='grayscale',
    image_size=(img_height, img_width),
    shuffle=True,
    seed=123, #same seed results in same validation test, good for testing bad for production
    validation_split=0.5,
    subset="validation",
)

Found 543 files belonging to 3 classes.
Using 271 files for validation.


In [6]:
#ds_train.shape

#def argument(x,y):
#    image = tf.image.random_brightness(x, max_delta=0.05)
#    return image, y

#ds_train = ds_train.map(argument)

#for Custom Loops
#for epochs in range(10):
#    for x, y in ds_train:
        #TRAIN HERE
#        pass

In [7]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.003),
    loss=[
        keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    ],
    metrics=["accuracy"],
)

model.fit(ds_train, epochs=10, verbose=2) #verbose is just a way to display the data, nothing to do with processing

Epoch 1/10
9/9 - 42s - loss: 216.6655 - accuracy: 0.3676
Epoch 2/10
9/9 - 16s - loss: 96.5570 - accuracy: 0.3676
Epoch 3/10
9/9 - 17s - loss: 42.5473 - accuracy: 0.3603
Epoch 4/10
9/9 - 16s - loss: 14.6938 - accuracy: 0.4228
Epoch 5/10
9/9 - 16s - loss: 3.7265 - accuracy: 0.5331
Epoch 6/10
9/9 - 16s - loss: 1.6059 - accuracy: 0.6618
Epoch 7/10
9/9 - 16s - loss: 0.9774 - accuracy: 0.7132
Epoch 8/10
9/9 - 16s - loss: 0.8124 - accuracy: 0.7721
Epoch 9/10
9/9 - 16s - loss: 0.4773 - accuracy: 0.8346
Epoch 10/10
9/9 - 16s - loss: 0.5771 - accuracy: 0.8015


In [8]:
#Code to flip images for dataset
#for images in path:
#    horizontal = image.transpose(PIL.Image.FLIP_LEFT_RIGHT)
#    horizontal.save("new_image")
#    verticle = image.transpose(PIL.Image.FLIP_TOP_BOTTOM)
#    verticle.save("new_image")
#    Horiz_Vert = verticle.transpose(PIL.Image.FLIP_LEFT_RIGHT)
#    Horiz_vert.save("new_image")


In [10]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

In [12]:
predictions = probability_model.predict(ds_validation)

In [29]:
i = 0
while i < 271:
    print(predictions[i])
    i = i + 1

[4.980440e-04 9.994923e-01 9.678382e-06]
[0.07076042 0.8943552  0.03488435]
[2.5010442e-03 9.9740309e-01 9.5790048e-05]
[1.0955288e-01 8.9042252e-01 2.4607740e-05]
[9.1887545e-05 3.6952647e-06 9.9990439e-01]
[0.8538883  0.13924834 0.00686341]
[0.10530547 0.8914162  0.00327829]
[8.0378973e-01 1.9606106e-01 1.4923440e-04]
[0.01633723 0.9811399  0.00252289]
[7.9311478e-01 2.0687346e-01 1.1684103e-05]
[0.8444076  0.15408887 0.00150348]
[1.9560628e-05 3.7174407e-06 9.9997675e-01]
[9.9820375e-01 1.3890885e-03 4.0724842e-04]
[0.7448837  0.25138122 0.00373498]
[4.1180258e-04 4.6899754e-06 9.9958354e-01]
[0.21915269 0.77991134 0.00093599]
[0.67846596 0.23842108 0.08311293]
[1.3145670e-06 1.0713203e-06 9.9999762e-01]
[9.9223197e-01 7.1395878e-03 6.2844827e-04]
[0.84200394 0.04904691 0.10894922]
[0.251757   0.01159475 0.7366482 ]
[0.0033275 0.8348078 0.1618647]
[1.3839791e-04 9.8516166e-01 1.4700039e-02]
[6.0208171e-04 9.9939775e-01 1.2276473e-07]
[3.0505341e-06 9.9999666e-01 2.9004718e-07]
[0.94

In [16]:
confMat = tf.math.confusion_matrix(ds_validation, predictions)

ValueError: Attempt to convert a value (<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int32)>) with an unsupported type (<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>) to a Tensor.